In [ ]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
# imports
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
# import base64
# from IPython.display import display, Markdown

# bokeh
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, ColumnDataSource
# from bokeh.models import HoverTool, CustomJS, ColumnDataSource, Span
# from bokeh.layouts import layout, column, gridplot
# from bokeh.layouts import row as bkrow
# from bokeh.palettes import all_palettes

# display graphics
output_notebook()

# My Gene's Expression in Type 2 Diabetes Transcriptomics Signatures

In [ ]:
%%appyter hide

{% do SectionField(
    name = 'INPUT',
    title = 'Input Parameters',
    subtitle = 'Fill in the species and gene of interest',
    img = 'mini_plot.png'
)%}

In [ ]:
%%appyter code_exec

{% set species = ChoiceField(
    name = 'species',
    label = 'Species of interest',
    choices = {
        'Human': 'human',
        'Mouse': 'mouse'
    },
    default = 'Human',
    description = 'Select the species of interest',
    section = 'INPUT'
)%}

{% set gene = StringField(
    name = 'gene',
    label = 'Gene of interest',
    default = 'SLC2A2',
    description = 'Enter the gene symbol of interest',
    section = 'INPUT'
)%}

In [ ]:
%%appyter code_exec
species = "{{species.value}}"
gene = "{{gene.value}}"

In [ ]:
# import preprocessed signature data
root_path = 'https://appyters.maayanlab.cloud/storage/Gene_Expression_T2D_Signatures/'
pval_df = pd.read_csv(root_path + f"all_{species}_pval.tsv", sep='\t', index_col=0)
fc_df = pd.read_csv(root_path + f"all_{species}_fc.tsv", sep='\t', index_col=0)

In [ ]:
# extract gene-specific data
comb_df = pd.DataFrame()
comb_df['sig'] = pval_df.columns.tolist()
comb_df['pval'] = pval_df.loc[gene].tolist()
comb_df['logpv'] = np.negative(np.log10(comb_df['pval']))
comb_df['fc'] = fc_df.loc[gene].tolist()

In [ ]:
# configure color scheme
red_map = cm.get_cmap('Reds_r')
red_norm = colors.Normalize(vmin=-0.25, vmax=1)
blue_map = cm.get_cmap('Blues_r')
blue_norm = colors.Normalize(vmin=-0.25, vmax=1)

def map_color(fc, pv):
    if fc < 0:
        return colors.to_hex(red_map(red_norm(pv)))
    elif fc == 0:
        return '#808080'
    else:
        return colors.to_hex(blue_map(blue_norm(pv)))

plot_colors = [map_color(r.fc, r.pval) for r in comb_df.itertuples()]
sizes = [12 if r.pval < 0.05 else 6 for r in comb_df.itertuples()]

In [ ]:
# generate data source
data_source = ColumnDataSource(
    data=dict(
        x = comb_df['fc'],
        y = comb_df['logpv'],
        sig = comb_df['sig'],
        pval = comb_df['pval'], 
        fc = comb_df['fc'], 
        colors = plot_colors, 
        sizes = sizes
    )
)
# create hover tooltip
tools = [
    ("Signature", "@sig"),
    ("P-Value", "@pval"),
    ("Fold Change", "@fc"),
]
# generate plot
plot = figure(
    plot_width=700,
    plot_height=500,
    tooltips=tools
)
plot.circle(
    'x', 'y', 
    size='sizes',
    alpha=0.7, 
    line_alpha=0,
    line_width=0.01, 
    source=data_source,
    fill_color='colors', 
    name='t2d_expression_volcano_plot'
)
plot.xaxis.axis_label = 'Fold Change'
plot.yaxis.axis_label = '-log10(p-value)'
plot.title.text = f"Expression of {gene} in Type 2 Diabetes Transcriptomics Signatures"
plot.title.align = 'center'
plot.title.text_font_size = '14px'

The volcano plot below positions each of the Type 2 Diabetes signatures according to the gene-specific fold change (x-position) and -log(p-value) (y-position). **Red** points indicate signatures where the fold change of the chosen gene was < 0, **blue** points indicate fold change > 0, and **gray** points indicate fold change == 0. Signatures where the specified gene was significantly expressed in either direction are denoted by darker color and larger point size. 

Hover over any point to display the corresponding signature name, the p-value, and the fold change. 

Use the toolbar on the right side of the plot to pan, zoom, or save the plot.

In [ ]:
show(plot)